In [47]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [48]:
class ClacSelfSimilarities(object):
    def __init__(self, img, region_size, patch_size, bin_size):
        self.img = img
        self.region_size = region_size
        self.patch_size = patch_size
        self.bin_size = bin_size
        self.theta, self.rho = self.cart2polar()
        self.bin = self.get_bin()
        self.alpha = 1/(85 ^ 2)

    def cart2polar(self):
        x, y = np.meshgrid(
            np.arange(self.region_size[1]), np.arange(self.region_size[0]))
        center_r = np.around(self.region_size/2).astype(int)
        theta = np.arctan2(x-center_r[1], y-center_r[0])
        rho = np.hypot(x-center_r[1], y-center_r[0])
        theta = theta*180/np.pi+180
        rho = np.log(rho)
        return theta, rho

    def get_bin(self):
        max_rho = np.max(self.rho)
        bin = {}
        m, n = np.arange(self.bin_size[0]), np.arange(self.bin_size[1])
        theta_low = m*24
        theta_up = (m+1)*24
        rho_low = max_rho*n/3
        rho_up = max_rho*(n+1)/3
        for i in range(self.bin_size[0]):
            for j in range(self.bin_size[1]):
                points = np.where((self.theta >= theta_low[i]) & (self.theta <= theta_up[i]) & (
                    self.rho >= rho_low[j]) & (self.rho <= rho_up[j]))
                bin[i, j] = points
        return bin

    def cal_ssd(self, patch, region, center_p):
        region_size = region.shape
        ssd = np.zeros((region_size[0], region_size[1]))
        for row in range(1+center_p[0], region_size[0]-center_p[0]):
            for col in range(1+center_p[1], region_size[1]-center_p[1]):
                ssd[row, col] = np.sum(
                    (patch-region[row-center_p[0]:row+center_p[0], col-center_p[1]:col+center_p[1], :])**2)
                ssd[row, col] = np.exp(-self.alpha*ssd[row, col])
        return ssd

    def get_self_sim_vec(self, region):
        self_similarities = np.zeros(self.bin_size[0]*self.bin_size[1])
        for i in range(self.bin_size[0]):
            for j in range(self.bin_size[1]):
                if len(region[self.bin[i, j]]) > 0:
                    self_similarities[i*self.bin_size[1] +
                                      j] = np.max(region[self.bin[i, j]])
                else:
                    self_similarities[i*self.bin_size[1]+j] = 0
        return self_similarities

    def cal_self_similarities(self):
        lab_image = cv2.cvtColor(self.img, cv2.COLOR_BGR2LAB)
        img_size = lab_image.shape
        vec_size = self.bin_size[0]*self.bin_size[1]
        self_similarities = np.zeros((img_size[0], img_size[1], vec_size))
        center_r = np.floor(self.region_size/2).astype(int)
        center_p = np.floor(self.patch_size/2).astype(int)

        for row in range(center_r[0]+1, img_size[0]-center_r[0]):
            for col in range(center_r[1]+1, img_size[1]-center_r[1]):
                patch = lab_image[row-center_p[0]:row +
                                  center_p[0], col-center_p[1]:col+center_p[1], :]
                region = lab_image[row-center_r[0]:row +
                                   center_r[0]+1, col-center_r[1]:col+center_r[1]+1, :]
                ssd_region = self.cal_ssd(patch, region, center_p)
                vec = self.get_self_sim_vec(ssd_region)
                self_similarities[row, col, :] = (
                    vec-np.min(vec))/(np.max(vec)-np.min(vec))
        return self_similarities

In [49]:
img = cv2.imread('images/1.jpg')
region_size = np.array([45, 37])
patch_size = np.array([5, 5])
bin_size = np.array([15, 3])
img = cv2.resize(img, (0, 0), fx=1/3, fy=1/3)
sim = ClacSelfSimilarities(img, region_size, patch_size, bin_size)

theta, rho = sim.theta, sim.rho
sim1 = sim.cal_self_similarities()
sim1

/tmp/ipykernel_359/2515413711.py:18: RuntimeWarning: divide by zero encountered in log
  rho = np.log(rho)


In [41]:
bin1[14, 1]

(array([13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 17, 17, 17,
        18, 18, 19]),
 array([18, 19, 20, 18, 19, 20, 21, 18, 19, 20, 21, 18, 19, 20, 18, 19, 20,
        18, 19, 19]))